In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from flask import Flask, request, jsonify, render_template
import json
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import os


model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')


input_file = 'arxiv-metadata-with-embeddings.json'


In [11]:
all_embeddings = []
all_sentences = []
metadata_index = []
paper_metadata = []

print("Loading embeddings and metadata...")
with open(input_file, 'r') as f:
    for i, line in enumerate(tqdm(f, desc="Processing JSONL")):
        paper = json.loads(line)
        if 'embeddings' in paper and 'sentences' in paper:
            embeddings = np.array(paper['embeddings'], dtype='float32')
            sentences = paper['sentences']
            all_embeddings.extend(embeddings)
            all_sentences.extend(sentences)
            metadata_index.extend([(i, j) for j in range(len(sentences))])
            paper_metadata.append({
                'id': paper.get('id', ''),
                'title': paper.get('title', 'Unknown'),
                'authors': paper.get('authors', []),
                'abstract': paper.get('abstract', ''),
                'doi': paper.get('doi', ''),
                'categories': paper.get('categories', []),
                'journal-ref': paper.get('journal-ref', '')
            })

Loading embeddings and metadata...


Processing JSONL: 2620981it [22:14, 1964.25it/s]


In [ ]:
paper

{'id': 'supr-con/9609004',
 'submitter': 'Masanori Ichioka',
 'authors': 'Naoki Enomoto, Masanori Ichioka and Kazushige Machida (Okayama Univ.)',
 'title': 'Ginzburg Landau theory for d-wave pairing and fourfold symmetric vortex\n  core structure',
 'comments': '12 pages including 8 eps figs, LaTeX with jpsj.sty & epsfig',
 'journal-ref': 'J. Phys. Soc. Jpn. 66, 204 (1997).',
 'doi': '10.1143/JPSJ.66.204',
 'report-no': None,
 'categories': 'supr-con cond-mat.supr-con',
 'license': None,
 'abstract': "  The Ginzburg Landau theory for d_{x^2-y^2}-wave superconductors is\nconstructed, by starting from the Gor'kov equation with including correction\nterms up to the next order of ln(T_c/T). Some of the non-local correction terms\nare found to break the cylindrical symmetry and lead to the fourfold symmetric\ncore structure, reflecting the internal degree of freedom in the pair\npotential. Using this extended Ginzburg Landau theory, we investigate the\nfourfold symmetric structure of the pa

: 

In [ ]:
all_embeddings = np.array(all_embeddings, dtype='float32')
print(f"Loaded {len(all_embeddings)} sentence embeddings from {len(paper_metadata)} papers.")
